In [1]:
import numpy as np
from matplotlib import cm

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os
import nibabel as nib
nibabel_dir = os.path.dirname(nib.__file__)
from nibabel import ecat
import pandas as pd
import torch
from torch.utils.data import Dataset

In [28]:
#file = '/home/andreas/Desktop/scientific_computing_project/bilder/02_PE2I_klinik_hermes_rCBF.v'
def read_data():
    data_path = 'Projektarbete_PE2I/Parameterbilder/ecat'

    #Collect all dicom images
    listFilesECAT = [] #create an empty list
    def load_scan2(path):
        for dirName, subdirList, fileList in os.walk(path):
            for filename in fileList:
                if ".v" in filename.lower(): #check wheter the file's ECAT
                    listFilesECAT.append(os.path.join(dirName, filename))

        return listFilesECAT

    # List of the path to the files in the specified directory
    patientList = load_scan2(data_path)
    patientList.sort() #SUVr 1st, rCBF 2nd

    # Get reference picture to get the proper sizes
    refImg = ecat.load(patientList[0]).get_frame(0)

    #Create an array to store the scans of all the patients
    images = np.zeros(( np.size(patientList),np.shape(refImg)[0],np.shape(refImg)[1],np.shape(refImg)[2]))
    for nr in range(np.size(patientList)):
        images[nr,:,:,:] = ecat.load(patientList[nr]).get_frame(0)
    images = images[...,np.newaxis]
    return images

In [29]:
class ImgDataset(Dataset):
    def __init__(self):
        self.labels = np.ones(56)
        self.samples = read_data() #Better to let samples be the pathway, more memory efficiency
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self,index):
        X = self.samples[index]
        Y = self.labels[index]
        return X,Y

In [30]:
dataset = ImgDataset()


In [31]:
dataset[:][0].shape

(56, 128, 128, 128, 1)

In [86]:
len(dataset)
test_data = dataset[32:40]
train_data = dataset[:32]

print(train_data[:][0].shape)
x_train = train_data[:][0]
y_train = train_data[:][1]

x_test = test_data[:][0]
y_test = test_data[:][1]

print(y_train.type)

(32, 128, 128, 128, 1)


AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [88]:
import keras

from keras.layers import Conv3D

model = keras.models.Sequential()

model.add(Conv3D(1, kernel_size=(3,3,3), input_shape = (128, 128, 128, 1)))

model.add(Flatten())



model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'Flatten' is not defined

In [87]:
model.fit(x_train,y_train,epochs=3)

ValueError: Error when checking target: expected conv3d_5 to have 5 dimensions, but got array with shape (32, 1)